In [ ]:
__author__ = 'Brian Merino <brian.merino@noirlab.edu>'
__version__ = '08252025' # yyyymmdd; version datestamp of this notebook
__keywords__ = ['AladinLite','des','skymapper','globular clusters','ipyaladin']

# Displaying images and catalogs with Aladin Lite v3

## Table of contents
* [Goals](#goals)
* [Summary](#summary)
* [Disclaimers and attribution](#disclaimer)
* [Imports and setup](#imports)
* [Start Aladin viewer](#Aladin)
* [Accessing the data](#Data)
* [Simple Image Access (SIA) Service Tool](#SIA)
* [Query the des_dr2 imaging data](#des_dr2)
* [Querying the Data Lab database](#Query)
* [Overlay table](#overlay_table)

<a class="anchor" id="goals"></a>
# Goals
Demonstrate how to display image and catalog data with the AladinLite Viewer in a notebook (using ipyaladin). The image and catalog data will be obtained through Data Lab's data services.

<a class="anchor" id="summary"></a>
# Summary
<a href="https://aladin.cds.unistra.fr/AladinLite/">Aladin Lite</a> is an interactive sky atlas that runs in your browser. Aladin can be used to explore the sky and has built-in functionality that makes it possible to overlay images onto the viewer and identify objects included in databases. <a href="https://github.com/cds-astro/ipyaladin">ipyaladin</a> was created to allow Jupyter Notebooks to utilize Aladin Lite's functionality. This notebook will demonstrate how to use ipyaladin to overlay fits files and survey data onto Aladin.

Visualizing surveys like this could help identify whether a source of interest has already been observed by a survey in the Data Lab, which could be helpful in preparing a telescope proposal. 


<a class="anchor" id="disclaimer"></a>
# Disclaimer & attribution
## Disclaimers
Note that using the Astro Data Lab constitutes your agreement with our minimal <a href="https://datalab.noirlab.edu/about/disclaimers">Disclaimers</a>.

## Acknowledgments
If you use Astro Data Lab in your published research, please include the text in your paper's Acknowledgments section:

This research uses services or data provided by the Astro Data Lab, which is part of the Community Science and Data Center (CSDC) Program of NSF NOIRLab. NOIRLab is operated by the Association of Universities for Research in Astronomy (AURA), Inc. under a cooperative agreement with the U.S. National Science Foundation.

If you use SPARCL jointly with the Astro Data Lab platform (via JupyterLab, command-line, or web interface) in your published research, please include this text below in your paper's Acknowledgments section:

This research uses services or data provided by the SPectra Analysis and Retrievable Catalog Lab (SPARCL) and the Astro Data Lab, which are both part of the Community Science and Data Center (CSDC) Program of NSF NOIRLab. NOIRLab is operated by the Association of Universities for Research in Astronomy (AURA), Inc. under a cooperative agreement with the U.S. National Science Foundation.

In either case please cite the following papers:

Data Lab concept paper: Fitzpatrick et al., "The NOAO Data Laboratory: a conceptual overview", SPIE, 9149, 2014, https://doi.org/10.1117/12.2057445

Astro Data Lab overview: Nikutta et al., "Data Lab - A Community Science Platform", Astronomy and Computing, 33, 2020, https://doi.org/10.1016/j.ascom.2020.100411

If you are referring to the Data Lab JupyterLab / Jupyter Notebooks, cite:

Juneau et al., "Jupyter-Enabled Astrophysical Analysis Using Data-Proximate Computing Platforms", CiSE, 23, 15, 2021, https://doi.org/10.1109/MCSE.2021.3057097
If publishing in a AAS journal, also add the keyword: \facility{Astro Data Lab}

And if you are using SPARCL, please also add \software{SPARCL} and cite:

Juneau et al., "SPARCL: SPectra Analysis and Retrievable Catalog Lab", Conference Proceedings for ADASS XXXIII, 2024 https://doi.org/10.48550/arXiv.2401.05576
The NOIRLab Library maintains lists of proper acknowledgments to use when publishing papers using the Lab's facilities, data, or services.

<a class="anchor" id="imports"></a>
# Imports and setup

In [ ]:
# Standard library
from getpass import getpass
import time

# Third-party libraries
import numpy as np
import matplotlib.pyplot as plt

import astropy.units as u
from astropy.wcs import WCS
from astropy.table import Table
from astropy.utils.data import download_file
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy.visualization import make_lupton_rgb, simple_norm
from pyvo.dal import sia
from ipyaladin import Aladin
from sidecar import Sidecar

# Data Lab
from dl import authClient as ac, queryClient as qc  

%matplotlib inline

<a class="anchor" id="Aladin"></a>
# Start Aladin Viewer

Let's start by opening an Aladin Lite viewer using SideCar. Running the following cell will open a new window to the right of where the notebook's cells are shown. 

Note: You do not need to use Sidecar to establish an Aladin Lite viewer session. If you were to just run the first line of the cell block, Aladin Lite would be opened below the code block. You could still interact with the viewer and utilize all of Aladin Lite's tools, but the session would remain below this code block, meaning you would need to scroll back up to this cell everytime you wanted to visit the viewer. 

In [ ]:
aladin = Aladin(full_screen=True)
with Sidecar(title="aladin_output",anchor='split-right'):
    display(aladin)

# If you would prefer the Aladin viewer to be displayed somewhere other than the right side,
# feel free to uncomment the following code to display Aladin at the bottom of the screen. 
#with Sidecar(title="aladin_output",anchor='split-bottom'):
#    display(aladin)

# The following line will make the notebook wait three seconds before continuing. 
# This will allow the notebook time to bring up the Aladin Viewer before executing
# the following commands. 
time.sleep(3)

Now that Aladin is running, let's center it on our target, the globular cluster NGC 1851. 

In [ ]:
#Center Aladin on NGC 1851
ra = 78.525 # in degrees
dec = -40.049 # in degrees
target_coord = SkyCoord(ra*u.deg,dec*u.deg)
aladin.target = target_coord

Next, let's adjust the field of view (fov) so we zoom in on the target. 

In [ ]:
aladin.fov = 0.2  # set field of view to 0.5 degrees

After running the previous cell, the Aladin viewer is ready to go. However, feel free to adjust the viewer to your liking. By interacting directly with the viewer, you can click on the '+' or '-' buttons located at the bottom left of the viewer to zoom in or out. Additionally, you can click and drag anywhere on the viewer's screen to adjust the center. You can also change the stretch of the image by right-clicking on the viewer and dragging your mouse around to adjust the brightness. 

<a class="anchor" id="Data"></a>
# Accessing the data

In this notebook, we are going to access two different datasets. First, we will use the Simple Image Access (SIA) service tool to access imaging data from The **D**ark **E**negery **S**urvey [(**DES**)](https://datalab.noirlab.edu/des/index.php), then we will use the query client to access the [**SkyMapper**](https://datalab.noirlab.edu/data/skymapper) fourth data release to identify sources near our target. 

Note: If you want to know where to find the URL used here, check out this [Astro Data Lab user manual page](https://datalab.noirlab.edu/docs/manual/UsingAstroDataLab/DataAccessInterfaces/SimpleImageAccessSIA/SimpleImageAccessSIA.html?highlight=sia) for the list of image collection URLs and other helpful information on using the SIA tool. 

### Authentication

Much of the functionality of Data Lab can be accessed without explicitly logging in (the service then uses an anonymous login). However, some capacities, such as saving the results of your queries to your virtual storage space, require a login (i.e., you will need a registered user account).

If you need to log in to Data Lab, un-comment the first line of code in the cell below and execute it:

In [ ]:
#token = ac.login(input("Enter user name: (+ENTER) "),getpass("Enter password: (+ENTER) "))
ac.whoAmI()

<a class="anchor" id="SIA"></a>
# SIA service tool

To access the SIA service tool, we need to establish the URL that points to the collection that holds the images we are interested in. For this example, we will access data from the des_dr2 image collection. 

In [ ]:
DEF_ACCESS_URL = "https://datalab.noirlab.edu/sia/des_dr2"
svc_des_dr2 = sia.SIAService(DEF_ACCESS_URL)

<a class="anchor" id="des_dr2"></a>
# Query the des_dr2 imaging data

Since we are working with data for the globular cluster NGC 1851 (RA = 78.525 degrees and Dec = -40.049 degrees), we will search for corresponding images in the des_dr2 image collection around these coordinates. Since the field of view (fov) parameter expects values in degrees, we do not need to use a large value.  

In [ ]:
fov = .2 # in degrees
imgTable_des_dr2 = svc_des_dr2.search((ra,dec), (fov/np.cos(dec*np.pi/180), fov), verbosity=2).to_table()
print(f"There are {len(imgTable_des_dr2)} matches. ")

Preview the table

In [ ]:
imgTable_des_dr2

Now that we have a table of images, we will select three different bands to create a false-color RGB image. For this example, we will choose images taken with the Y, Z, and I filters. The reddest filter, Y, will be used for our red image; the bluest, I, for our blue image; and the Z filter, which falls between both, for our green image. Multiple images are taken with each filter. Here, we will select the first image for each filter as it appears in the table.

Feel free to change the row used to select the image to see how the RGB image changes. 

To access the images, we can use their access URLs to download them directly into our notebook.

In [ ]:
def download_image(url):
    filename = download_file(url, cache=True, show_progress=False, timeout=120)
    hdu      = fits.open(filename)[0]
    image    = hdu.data
    hdr      = hdu.header
    wcs      = WCS(hdr)

    return image,wcs

In [ ]:
# Image 1 - Y band
row_1 = imgTable_des_dr2[0]
url_1 = row_1['access_url']
image_1,wcs_1 = download_image(url_1)

# Image 2 - Z band
row_2 = imgTable_des_dr2[27]
url_2 = row_2['access_url']
image_2,wcs_2 = download_image(url_2)

# Image 3 - I band
row_3 = imgTable_des_dr2[15]
url_3 = row_3['access_url']
image_3,wcs_3 = download_image(url_3)

With all three images opened lets plot them using matplotlib to get an idea of what they look like. 

In [ ]:
fig = plt.figure(figsize=(14, 10))

# Subplot 1
ax1 = fig.add_subplot(1, 3, 1, projection=wcs_1)
ax1.set_title('Image 1 - y band')
im1 = ax1.imshow(image_1, origin='lower', cmap='Greys_r', vmin=image_1.min(), vmax=image_1.min()+(image_1.max()-image_1.min())/100.)

# Subplot 2
ax2 = fig.add_subplot(1, 3, 2, projection=wcs_2)
ax2.set_title('Image 2 - z band')
im2 = ax2.imshow(image_2, origin='lower', cmap='Greys_r', vmin=image_2.min(), vmax=image_2.min()+(image_2.max()-image_2.min())/70.)

# Subplot 3
ax3 = fig.add_subplot(1, 3, 3, projection=wcs_3)
ax3.set_title('Image 3 - i band')
im3 = ax3.imshow(image_3, origin='lower', cmap='Greys_r', vmin=image_3.min(), vmax=np.abs(image_3.min())+(image_3.max()-np.abs(image_3.min()))/25.)

plt.tight_layout()
plt.show()

Here we can see how NGC 1851 looks in the y, z, and i bands. The redest band, the y band, shows the older stellar population. The bluer bands, specifically the z and i bands, show more stars, with the z band displaying the most stars. This tells us that the majority of stars in this cluster peak between 850 and 950 nm. 

<a class="anchor" id="overlay_image"></a>
# Overlay images on Aladin

After plotting all three images, we can verify that they are of the same source, but they are not identical. 

Now, we can use the add_fits() function to overlay the images onto Aladin. By default, Aladin sets the opacity of each image to 1.0, which means only the last image added will be visible. To remedy this, we will manually vary the opacities so that the second and third images are slightly transparent so that all three images can be seen.

Note: after adding the images to Aladin, the fov will change automatically. If you wish to adjust the fov, use the aladin.fov() command that we used earlier in this notebook. 

In [ ]:
#Add Image_1 to Aladin
aladin.add_fits(url_1,name='Y band',opacity=1.0,colormap='red')

#Add image_2 to Aladin
aladin.add_fits(url_2,name='Z band',opacity=0.5,colormap='green')

#Add image_2 to Aladin
aladin.add_fits(url_3,name='I band',opacity=0.25,colormap='blue')

<a class="anchor" id="query"></a>
# Query Data Lab

Now that we know how to work with images, let's query the data lab to get data for individual stars in and around our target. We will use the SkyMapper data release 4 table for this notebook because our target falls within the survey's footprint. We only need the ra_img, dec_img, and object_id columns for this example. 

Since we already know the coordinates of our target, we can perform a cone search centered around the globular cluster. Note that we set the FOV to be 0.2 degrees when working with images because we wanted to find any images around our target, so we cast a wide net. For this query, we will need to narrow our FOV because SkyMapper DR4 contains data for **many** sources near our target. Using the same FOV as the image search would return 44,470 sources! Many of which have nothing to do with our target. If we narrow our FOV by just a factor of 3, the radius of the cone search becomes about twice as large as the diameter of the globular cluster. This query will still return a lot of sources, but this time, most of them are closer to our target. 

In [ ]:
sql = '''SELECT ra_img, decl_img, mag_psf, object_id
            FROM skymapper_dr4.photometry
            WHERE Q3C_RADIAL_QUERY(ra_img,decl_img,{0},{1},{2})
'''.format(ra, dec, fov/6)
df = qc.query(sql=sql, fmt='pandas')
df

The SkyMapper survey contains tens of thousands of data points. If we were to overlay the data points as they are, the image would be obscured by a blanket of circular regions, completely blocking our view of the globular cluster. To resolve this, we will apply a magnitude cutoff to reduce the number of regions we will overlay onto Aladin. 

Let's establish a magnitude cutoff so that only sources with magnitudes less than 16 are displayed. 

In [ ]:
mag_filt = df['mag_psf'] < 16
len(df[mag_filt])

Let's see what the individual points look like when plotted over our first image. 

In [ ]:
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(1, 1, 1, projection=wcs_1)
ax.imshow(image_1, cmap='gray', vmin=image_1.min(), vmax=image_1.min()+(image_1.max()-image_1.min())/100.)
ax.scatter(df['ra_img'][mag_filt], df['decl_img'][mag_filt], transform=ax.get_transform('icrs'),
           edgecolor='green', facecolor='none', linewidths=1, label='SkyMapper')

circle  = plt.Circle((825,1375), 490, color='red', fill=False, label='FOV - %s"'%(round(fov/6,2)))
ax.add_patch(circle)

plt.legend()
plt.show()

# Rewrite this section

With the data points overplotted onto our image, you can see that many of the stars on the outskirs of the cluster appear in the center of the green circles, but there are few green circles in the center of the cluster where most of the stars are. 

<a class="anchor" id="overlay_table"></a>
# Overlay Table

Now, we are just about ready to overlay the data from this table on Aladin. But before we can overlay our table, we must first convert it from a 
pandas data frame to a table. 

Finally, we can use Aladin's built-in add_table() function to add the points to our Aladin viewer. 

In [ ]:
t = Table.from_pandas(df[mag_filt])

aladin.add_table(
    t,
    shape='circle',
    source_size=15,
    color='lightblue')